## 1. Biblioteki

In [2]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import os
import talos as ta
from talos.model.early_stopper import early_stopper
from sklearn.preprocessing import StandardScaler

%load_ext tensorboard
import tensorflow_addons as tfa
import keras
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.metrics import Recall, Precision
import seaborn as sns

from sklearn.model_selection import train_test_split


import tensorflow as tf

from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score
from tensorflow.keras.callbacks import EarlyStopping
from focal_loss import BinaryFocalLoss

In [3]:
"""
Define our custom loss function.
"""
import numpy as np
from keras import backend as K
import tensorflow as tf



def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        y_true = tf.cast(y_true, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss

    return binary_focal_loss_fixed


def categorical_focal_loss(alpha, gamma=2.):
    """
    Softmax version of focal loss.
    When there is a skew between different categories/labels in your data set, you can try to apply this function as a
    loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
      categories/labels, the size of the array needs to be consistent with the number of classes.
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    alpha = np.array(alpha, dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed



## 2. Dane

In [4]:
train_df=pd.read_csv("D:/STUDIA/ROK_II/Magisterka/Modele/Dane pierwotne/Dane_do_uczenia_M.csv", encoding="utf-8")
del train_df['Unnamed: 0']

In [5]:
train_df.head()

,ENSG00000212232.1,ENSG00000238741.1,ENSG00000252481.1,ENSG00000239002.3,ENSG00000212443.1,ENSG00000274012.1,ENSG00000252010.1,ENSG00000202198.1,ENSG00000251791.1,ENSG00000202058.1,...,ENSG00000241475.1,ENSG00000274618.1,ENSG00000227293.1,ENSG00000253526.1,ENSG00000270654.1,ENSG00000271394.1,ENSG00000265423.1,ENSG00000253165.1,ENSG00000201901.1,scale
0,27.356617,31.768974,27.356617,5.294829,8.824715,6.645010e+02,2.647415,4.235863e+02,7.059772,16.766959,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,M0
1,8.695633,1.086954,6.521724,2.173908,0.000000,1.758692e+03,1.086954,5.434770e+01,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,M0
2,20.185177,77.002713,7.475992,4.485595,8.971190,2.775836e+03,2.242797,2.609121e+02,6.728392,10.466388,...,0.000000,0.747599,0.0,0.747599,0.0,0.747599,0.000000,0.0,0.000000,M0
3,18.157930,17.431612,6.536855,2.178952,2.905269,2.338741e+02,3.631586,6.827382e+01,1.452634,1.452634,...,0.726317,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,M0
4,29912.168049,21631.677176,9554.333460,20332.131551,5136.495208,1.255850e+06,19221.760289,1.926818e+06,5198.182500,2655.637935,...,0.000000,953.068666,0.0,20.562431,0.0,269.367843,64.771657,0.0,123.374585,M0


In [6]:
train_df['scale'].loc[(train_df['scale'] == 'M0')] = 0
train_df['scale'].loc[(train_df['scale'] == 'M1')] = 1

In [7]:
train_df=train_df.loc[(train_df['scale']!='MX')]

In [8]:
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42,stratify=train_df['scale'])
#test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=42,stratify=test_df['scale'])


train_label=train_df['scale']
test_label=test_df['scale']
#val_label=val_df['scale']
del train_df['scale']
del test_df['scale']
#del val_df['scale']


In [9]:
len(list(train_df))

110

In [10]:
train_label.shape

(568,)

In [11]:
test_label.shape

(142,)

In [12]:
#val_label.shape

## 1.2 Standaryzacja

In [13]:
scaler = StandardScaler()
train_df=scaler.fit_transform(train_df)
test_df=scaler.fit_transform(test_df)
#val_df=scaler.fit_transform(val_df)


In [14]:
train_df

array([[-0.23532639, -0.28536286, -0.29189695, ..., -0.20273415,
        -0.13658188, -0.25716709],
       [-0.23488914, -0.28412701, -0.28766847, ..., -0.20273415,
        -0.13658188, -0.25716709],
       [-0.23513615, -0.28194648, -0.28119849, ..., -0.1127296 ,
        -0.13658188, -0.18291094],
       ...,
       [-0.23560858, -0.28411483, -0.29158105, ..., -0.20273415,
        -0.07110588, -0.25716709],
       [13.92769305,  3.11477926,  6.81130447, ..., -0.05473062,
        -0.13658188,  1.2895196 ],
       [-0.23495945, -0.28021459, -0.28164044, ..., -0.20273415,
        -0.07405669, -0.25716709]])

In [15]:
train_df.shape

(568, 110)

In [16]:
train_label=np.asarray(train_label).astype(np.int)
test_label=np.asarray(test_label).astype(np.int)
#val_label=np.asarray(val_label).astype(np.int)

In [17]:
train_label

array([0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
train_label.shape

(568,)

In [19]:
train_label=train_label.reshape(-1,1)
test_label=test_label.reshape(-1,1)
#val_label=val_label.reshape(-1,1)

In [20]:
test_label.shape

(142, 1)

# 2 Moduł TALOS

## 2.1 Słownik parametrów do wypróbowania

In [ ]:
## Dobor Gammy i alfy

In [78]:
p = {
    'first_neuron':[110], 
     'hidden_neuron':[100],

     'hidden_layers':[3],  

     
    'epochs': [100000], # never touch it


    'last_activation': ['sigmoid'], #never touch it


    'batch_size': [32],

    'lr':[0.001],
    
    'kernel_regularizer_l1':[0.0001],#[0,0.001,0.0001],
    'kernel_regularizer_l2':[0.0001],#[0,0.001,0.0001],
    'bias_regularizer':[0.0001],#[0,0.001,0.0001],
    'activity_regularizer':[0.0001],#[0,0.001,0.0001],

#    'batc_normalization':[False,True],
#    'dropout': [0,0.2,0.4],
    'dropout': [0],
    
    #'optimizer': ['rmsprop','adam','adadelta','adamax','nadam','adagrad'],
    #'kernel_initializer': ['orthogonal','identity','zeros','ones','uniform'],
    'kernel_initializer': ['orthogonal'],
    #'activation_layer':['sigmoid','tanh','selu','elu','relu'],
    'activation_layer':['relu'],
    #'batc_normalization':[False,True]
    'batc_normalization':[False],
    'alpha':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
     

    'gamma':[0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
    
    
    
}

In [79]:

def numerai_model(x_train, y_train, x_val, y_val, params):
    
    print(params)

    model = Sequential()
    
    ## initial layer
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation='relu',
               
                    kernel_initializer = params['kernel_initializer'],
                    kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                                l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                   ))
    if params['batc_normalization']==True:
        model.add(BatchNormalization())
        
    if params['dropout']!=0:
        model.add(Dropout(params['dropout']))
    
    ## hidden layers
    for i in range(params['hidden_layers']):
        print (f"adding layer {i+1}")
        model.add(Dense(params['hidden_neuron'], activation='relu',
                    kernel_initializer=params['kernel_initializer'],
                        kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                                    l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                       ))
        if params['batc_normalization']==True:
            model.add(BatchNormalization())
            
        if params['dropout']!=0:
            model.add(Dropout(params['dropout']))
    
    
    ## final layer
    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer'],
                   kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                               l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                   ))
    
    model.compile(loss=[binary_focal_loss(alpha=params['alpha'], gamma=params['gamma'])],
                  optimizer=tf.keras.optimizers.Adamax(learning_rate=params['lr']),
                  metrics=[tfa.metrics.FBetaScore(num_classes=1, beta=1.0, threshold=0.5),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0,
                        callbacks = [
                                     EarlyStopping(monitor='val_loss',
                                        min_delta=0.01,
                                        patience=50, mode='min',verbose=1,
                                                      restore_best_weights=True)
                                    ] #,ta.live(),
                        )

    return history, model

## 2.3 Przeprowadzam skan, używając parametrów i funkcji wyżej


In [80]:

t = ta.Scan(x=train_df, y=train_label,
            x_val=test_df, y_val=test_label,
            model=numerai_model,
            params=p,  
            experiment_name='Predykcja klasy M - podejscie z Focal-loss (nowe)')


  0%|                                                                                          | 0/110 [00:00<?, ?it/s]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.1, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 0, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.001}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, 


  1%|▋                                                                                 | 1/110 [00:12<22:38, 12.46s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.1, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 0.5, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.001}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning


  2%|█▍                                                                                | 2/110 [00:23<21:24, 11.90s/it]

...


 98%|██████████████████████████████████████████████████████████████████████████████▌ | 108/110 [16:39<00:14,  7.40s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 1, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 4.5, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.001}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, 


 99%|███████████████████████████████████████████████████████████████████████████████▎| 109/110 [16:45<00:06,  6.99s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 1, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 5, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.001}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, de


100%|████████████████████████████████████████████████████████████████████████████████| 110/110 [16:53<00:00,  9.21s/it]


In [67]:
df = pd.read_csv('D:/STUDIA/ROK_II/Magisterka/Modele/Dane pierwotne/M/Predykcja klasy M - podejscie z Focal-loss (nowe)/050522211619.csv')

In [68]:
stare_wart=df['val_fbeta_score']
nowe_wart=[]
for x in stare_wart:
    wart=x.replace('[','')
    wart=wart.replace(']','')
    wart=float(wart)
    nowe_wart.append(wart)

In [69]:
nowe_wart

[0.22222222,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.27777776,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267,
 0.26829267]

In [70]:
df['val_fbeta_score']=nowe_wart

In [71]:
df=df.sort_values('val_loss',ascending=True)

In [72]:
df.columns

Index(['round_epochs', 'loss', 'fbeta_score', 'precision', 'recall',
       'val_loss', 'val_fbeta_score', 'val_precision', 'val_recall',
       'activation_layer', 'activity_regularizer', 'alpha',
       'batc_normalization', 'batch_size', 'bias_regularizer', 'dropout',
       'epochs', 'first_neuron', 'gamma', 'hidden_layers', 'hidden_neuron',
       'kernel_initializer', 'kernel_regularizer_l1', 'kernel_regularizer_l2',
       'last_activation', 'lr'],
      dtype='object')

In [73]:
df

,round_epochs,loss,fbeta_score,precision,recall,val_loss,val_fbeta_score,val_precision,val_recall,activation_layer,...,epochs,first_neuron,gamma,hidden_layers,hidden_neuron,kernel_initializer,kernel_regularizer_l1,kernel_regularizer_l2,last_activation,lr
31,294,-27.039412,[0.26299694],0.151408,1.000000,-26.926886,0.268293,0.154930,1.000000,relu,...,100000,110,6,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
28,306,-27.038244,[0.26299694],0.151408,1.000000,-26.926428,0.268293,0.154930,1.000000,relu,...,100000,110,0,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
30,282,-27.036730,[0.26299694],0.151408,1.000000,-26.924812,0.268293,0.154930,1.000000,relu,...,100000,110,4,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
32,319,-27.035040,[0.26299694],0.151408,1.000000,-26.923359,0.268293,0.154930,1.000000,relu,...,100000,110,8,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
34,348,-27.035099,[0.26299694],0.151408,1.000000,-26.922380,0.268293,0.154930,1.000000,relu,...,100000,110,12,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
33,280,-27.033897,[0.26299694],0.151408,1.000000,-26.921867,0.268293,0.154930,1.000000,relu,...,100000,110,10,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
29,299,-27.032303,[0.26299694],0.151408,1.000000,-26.920349,0.268293,0.154930,1.000000,relu,...,100000,110,2,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
24,209,-13.509267,[0.26299694],0.151408,1.000000,-13.453563,0.268293,0.154930,1.000000,relu,...,100000,110,6,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
26,252,-13.509299,[0.26299694],0.151408,1.000000,-13.453326,0.268293,0.154930,1.000000,relu,...,100000,110,10,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
27,241,-13.507066,[0.26299694],0.151408,1.000000,-13.451137,0.268293,0.154930,1.000000,relu,...,100000,110,12,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01


In [74]:
df=df.reindex(columns=['val_fbeta_score','loss', 'fbeta_score',
       'precision', 'recall', 'val_loss', 
      'val_precision',
       'val_recall', 'gamma','alpha'])

In [75]:
df

,val_fbeta_score,loss,fbeta_score,precision,recall,val_loss,val_precision,val_recall,gamma,alpha
31,0.268293,-27.039412,[0.26299694],0.151408,1.000000,-26.926886,0.154930,1.000000,6,3.0
28,0.268293,-27.038244,[0.26299694],0.151408,1.000000,-26.926428,0.154930,1.000000,0,3.0
30,0.268293,-27.036730,[0.26299694],0.151408,1.000000,-26.924812,0.154930,1.000000,4,3.0
32,0.268293,-27.035040,[0.26299694],0.151408,1.000000,-26.923359,0.154930,1.000000,8,3.0
34,0.268293,-27.035099,[0.26299694],0.151408,1.000000,-26.922380,0.154930,1.000000,12,3.0
33,0.268293,-27.033897,[0.26299694],0.151408,1.000000,-26.921867,0.154930,1.000000,10,3.0
29,0.268293,-27.032303,[0.26299694],0.151408,1.000000,-26.920349,0.154930,1.000000,2,3.0
24,0.268293,-13.509267,[0.26299694],0.151408,1.000000,-13.453563,0.154930,1.000000,6,2.0
26,0.268293,-13.509299,[0.26299694],0.151408,1.000000,-13.453326,0.154930,1.000000,10,2.0
27,0.268293,-13.507066,[0.26299694],0.151408,1.000000,-13.451137,0.154930,1.000000,12,2.0


In [76]:
df=df[df['val_fbeta_score']!=0]

In [77]:
df

,val_fbeta_score,loss,fbeta_score,precision,recall,val_loss,val_precision,val_recall,gamma,alpha
31,0.268293,-27.039412,[0.26299694],0.151408,1.000000,-26.926886,0.154930,1.000000,6,3.0
28,0.268293,-27.038244,[0.26299694],0.151408,1.000000,-26.926428,0.154930,1.000000,0,3.0
30,0.268293,-27.036730,[0.26299694],0.151408,1.000000,-26.924812,0.154930,1.000000,4,3.0
32,0.268293,-27.035040,[0.26299694],0.151408,1.000000,-26.923359,0.154930,1.000000,8,3.0
34,0.268293,-27.035099,[0.26299694],0.151408,1.000000,-26.922380,0.154930,1.000000,12,3.0
33,0.268293,-27.033897,[0.26299694],0.151408,1.000000,-26.921867,0.154930,1.000000,10,3.0
29,0.268293,-27.032303,[0.26299694],0.151408,1.000000,-26.920349,0.154930,1.000000,2,3.0
24,0.268293,-13.509267,[0.26299694],0.151408,1.000000,-13.453563,0.154930,1.000000,6,2.0
26,0.268293,-13.509299,[0.26299694],0.151408,1.000000,-13.453326,0.154930,1.000000,10,2.0
27,0.268293,-13.507066,[0.26299694],0.151408,1.000000,-13.451137,0.154930,1.000000,12,2.0


## Part 2

In [81]:
df = pd.read_csv('D:/STUDIA/ROK_II/Magisterka/Modele/Dane pierwotne/M/Predykcja klasy M - podejscie z Focal-loss (nowe)/050522215522.csv')

In [97]:
stare_wart=df['val_fbeta_score']
nowe_wart=[]
for x in stare_wart:
    wart=x.replace('[','')
    wart=wart.replace(']','')
    wart=float(wart)
    nowe_wart.append(wart)

In [98]:
df['val_fbeta_score']=nowe_wart

In [112]:
df=df.sort_values('val_loss',ascending=False)

In [106]:
df

,val_fbeta_score,loss,fbeta_score,precision,recall,val_loss,val_precision,val_recall,gamma,alpha
109,0.268293,0.001101,[0.26299694],0.151408,1.000000,0.001101,0.154930,1.000000,5.0,1.0
108,0.268293,0.001361,[0.26299694],0.151408,1.000000,0.001361,0.154930,1.000000,4.5,1.0
107,0.268293,0.001398,[0.26299694],0.151408,1.000000,0.001395,0.154930,1.000000,4.0,1.0
106,0.268293,0.001443,[0.26299694],0.151408,1.000000,0.001444,0.154930,1.000000,3.5,1.0
103,0.268293,0.002017,[0.26299694],0.151408,1.000000,0.002019,0.154930,1.000000,2.0,1.0
104,0.268293,0.002037,[0.26299694],0.151408,1.000000,0.002028,0.154930,1.000000,2.5,1.0
102,0.268293,0.002070,[0.26299694],0.151408,1.000000,0.002071,0.154930,1.000000,1.5,1.0
105,0.268293,0.002143,[0.26299694],0.151408,1.000000,0.002133,0.154930,1.000000,3.0,1.0
100,0.268293,0.003228,[0.26299694],0.151408,1.000000,0.003222,0.154930,1.000000,0.5,1.0
101,0.268293,0.003385,[0.26299694],0.151408,1.000000,0.003366,0.154930,1.000000,1.0,1.0


In [114]:
df=df.reindex(columns=['val_fbeta_score', 'fbeta_score',
       'precision', 'recall', 'val_loss', 'loss',
      'val_precision',
       'val_recall', 'gamma','alpha'])

In [108]:
df=df[df['val_fbeta_score']!=0]

In [117]:
df=df[df['recall']!=1]

In [119]:
df=df[df['precision']!=1]

In [120]:
df

,val_fbeta_score,fbeta_score,precision,recall,val_loss,loss,val_precision,val_recall,gamma,alpha
55,0.142857,[0.29126212],0.882353,0.174419,0.276685,0.213429,0.333333,0.090909,0.0,0.6
66,0.200000,[0.2962963],0.727273,0.186047,0.266233,0.220536,0.375000,0.136364,0.0,0.7
44,0.153846,[0.25742576],0.866667,0.151163,0.259258,0.207913,0.500000,0.090909,0.0,0.5
77,0.214286,[0.42152467],0.343066,0.546512,0.248677,0.197131,0.500000,0.136364,0.0,0.8
67,0.193548,[0.35200003],0.564103,0.255814,0.219989,0.151576,0.333333,0.136364,0.5,0.7
56,0.148148,[0.26666665],0.736842,0.162791,0.209560,0.155029,0.400000,0.090909,0.5,0.6
88,0.279070,[0.31365314],0.186404,0.988372,0.190441,0.140976,0.168224,0.818182,0.0,0.9
78,0.187500,[0.38333333],0.298701,0.534884,0.161941,0.139186,0.300000,0.136364,0.5,0.8
68,0.206897,[0.22000001],0.785714,0.127907,0.139918,0.111559,0.428571,0.136364,1.0,0.7
89,0.289855,[0.3113553],0.184783,0.988372,0.120481,0.100924,0.172414,0.909091,0.5,0.9


In [ ]:
## part 3
alpha 0.9 - 0.6
gamma!=0, powyzej 1-2

In [121]:
p = {
    'first_neuron':[110], 
     'hidden_neuron':[100],

     'hidden_layers':[3],  

     
    'epochs': [100000], # never touch it


    'last_activation': ['sigmoid'], #never touch it


    'batch_size': [32],

    'lr':[0.01],
    
    'kernel_regularizer_l1':[0.0001],#[0,0.001,0.0001],
    'kernel_regularizer_l2':[0.0001],#[0,0.001,0.0001],
    'bias_regularizer':[0.0001],#[0,0.001,0.0001],
    'activity_regularizer':[0.0001],#[0,0.001,0.0001],

#    'batc_normalization':[False,True],
#    'dropout': [0,0.2,0.4],
    'dropout': [0],
    
    #'optimizer': ['rmsprop','adam','adadelta','adamax','nadam','adagrad'],
    #'kernel_initializer': ['orthogonal','identity','zeros','ones','uniform'],
    'kernel_initializer': ['orthogonal'],
    #'activation_layer':['sigmoid','tanh','selu','elu','relu'],
    'activation_layer':['relu'],
    #'batc_normalization':[False,True]
    'batc_normalization':[False],
    'alpha':[0.6,0.65,0.7,0.75,0.8,0.85,0.9],
    'gamma':[1,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2]
    
    
    
}

In [122]:

def numerai_model(x_train, y_train, x_val, y_val, params):
    
    print(params)

    model = Sequential()
    
    ## initial layer
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation='relu',
               
                    kernel_initializer = params['kernel_initializer'],
                    kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                                l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                   ))
    if params['batc_normalization']==True:
        model.add(BatchNormalization())
        
    if params['dropout']!=0:
        model.add(Dropout(params['dropout']))
    
    ## hidden layers
    for i in range(params['hidden_layers']):
        print (f"adding layer {i+1}")
        model.add(Dense(params['hidden_neuron'], activation='relu',
                    kernel_initializer=params['kernel_initializer'],
                        kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                                    l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                       ))
        if params['batc_normalization']==True:
            model.add(BatchNormalization())
            
        if params['dropout']!=0:
            model.add(Dropout(params['dropout']))
    
    
    ## final layer
    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer'],
                   kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                               l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                   ))
    
    model.compile(loss=[binary_focal_loss(alpha=params['alpha'], gamma=params['gamma'])],
                  optimizer=tf.keras.optimizers.Adamax(learning_rate=params['lr']),
                  metrics=[tfa.metrics.FBetaScore(num_classes=1, beta=1.0, threshold=0.5),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0,
                        callbacks = [
                                     EarlyStopping(monitor='val_loss',
                                        min_delta=0.01,
                                        patience=50, mode='min',verbose=1,
                                                      restore_best_weights=True)
                                    ] #,ta.live(),
                        )

    return history, model

In [123]:
t = ta.Scan(x=train_df, y=train_label,
            x_val=test_df, y_val=test_label,
            model=numerai_model,
            params=p,  
            experiment_name='Predykcja klasy M - podejscie z Focal-loss (nowe)')


  0%|                                                                                           | 0/77 [00:00<?, ?it/s]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.6, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 1, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.01}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, d

  8%|██████                                                                      | 2/25 [1:41:19<19:25:10, 3039.61s/it]

  1%|█                                                                                  | 1/77 [00:06<07:54,  6.25s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.6, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 1.1, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.01}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning,


  3%|██▏                                                                                | 2/77 [00:11<07:23,  5.91s/it]

...


 97%|███████████████████████████████████████████████████████████████████████████████▊  | 75/77 [07:15<00:10,  5.46s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.9, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 1.9, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.01}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning,


 99%|████████████████████████████████████████████████████████████████████████████████▉ | 76/77 [07:20<00:05,  5.44s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.9, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 2, 'hidden_layers': 3, 'hidden_neuron': 100, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'lr': 0.01}
adding layer 1
adding layer 2
adding layer 3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, d


100%|██████████████████████████████████████████████████████████████████████████████████| 77/77 [07:25<00:00,  5.79s/it]


In [139]:
df = pd.read_csv('D:/STUDIA/ROK_II/Magisterka/Modele/Dane pierwotne/M/Predykcja klasy M - podejscie z Focal-loss (nowe)/050522222938.csv')

In [140]:
df

,round_epochs,loss,fbeta_score,precision,recall,val_loss,val_fbeta_score,val_precision,val_recall,activation_layer,...,epochs,first_neuron,gamma,hidden_layers,hidden_neuron,kernel_initializer,kernel_regularizer_l1,kernel_regularizer_l2,last_activation,lr
0,80,0.108194,[0.13043478],1.000000,0.069767,0.124587,[0.],0.000000,0.000000,relu,...,100000,110,1.0,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
1,68,0.101885,[0.15053762],1.000000,0.081395,0.120106,[0.],0.000000,0.000000,relu,...,100000,110,1.1,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
2,78,0.095572,[0.15053762],1.000000,0.081395,0.117651,[0.16000001],0.666667,0.090909,relu,...,100000,110,1.2,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
3,98,0.088541,[0.06741573],1.000000,0.034884,0.106944,[0.],0.000000,0.000000,relu,...,100000,110,1.3,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
4,81,0.084127,[0.12903227],0.857143,0.069767,0.096816,[0.16000001],0.666667,0.090909,relu,...,100000,110,1.4,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,76,0.052518,[0.26299694],0.151408,1.000000,0.052903,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.6,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
73,63,0.049284,[0.26299694],0.151408,1.000000,0.049682,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.7,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
74,55,0.046674,[0.26299694],0.151408,1.000000,0.047030,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.8,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01
75,63,0.043638,[0.26299694],0.151408,1.000000,0.043978,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.9,3,100,orthogonal,0.0001,0.0001,sigmoid,0.01


In [141]:
df=df.reindex(columns=['val_fbeta_score', 'fbeta_score',
       'precision', 'recall', 'val_loss', 'loss',
      'val_precision',
       'val_recall', 'gamma','alpha'])

In [150]:
df=df[df['val_fbeta_score']!=0]

In [147]:
df=df[df['recall']!=1]

In [148]:
df=df[df['precision']!=1]

In [143]:
df=df.sort_values('val_loss',ascending=False)

In [144]:
stare_wart=df['val_fbeta_score']
nowe_wart=[]
for x in stare_wart:
    wart=x.replace('[','')
    wart=wart.replace(']','')
    wart=float(wart)
    nowe_wart.append(wart)

In [145]:
df['val_fbeta_score']=nowe_wart

In [153]:
df=df[df['val_fbeta_score']>=0.2]

In [154]:
df

,val_fbeta_score,fbeta_score,precision,recall,val_loss,loss,val_precision,val_recall,gamma,alpha
45,0.222222,[0.2201835],0.521739,0.139535,0.113385,0.088619,0.600000,0.136364,1.1,0.80
55,0.281690,[0.3068592],0.181624,0.988372,0.100955,0.086858,0.166667,0.909091,1.0,0.85
56,0.285714,[0.30601093],0.181425,0.976744,0.096065,0.081503,0.169492,0.909091,1.1,0.85
57,0.281690,[0.30852994],0.182796,0.988372,0.095733,0.075617,0.166667,0.909091,1.2,0.85
47,0.206897,[0.17821783],0.600000,0.104651,0.090992,0.079640,0.428571,0.136364,1.3,0.80
58,0.289855,[0.31365314],0.186404,0.988372,0.088710,0.070545,0.172414,0.909091,1.3,0.85
66,0.277778,[0.3041145],0.179704,0.988372,0.083474,0.073808,0.163934,0.909091,1.0,0.90
49,0.214286,[0.18181817],0.692308,0.104651,0.082003,0.069394,0.500000,0.136364,1.5,0.80
59,0.285714,[0.3099631],0.184211,0.976744,0.077891,0.066723,0.169492,0.909091,1.4,0.85
50,0.222222,[0.20833331],0.258621,0.174419,0.075508,0.064734,0.600000,0.136364,1.6,0.80


## Part 3

In [157]:
p = {
    'first_neuron':[50,110], 
     'hidden_neuron':[50,150],

     'hidden_layers':[1,3,5],  

     
    'epochs': [100000], # never touch it


    'last_activation': ['sigmoid'], #never touch it


    'batch_size': [32],

    #'lr':[0.01],
    
    'kernel_regularizer_l1':[0.0001],#[0,0.001,0.0001],
    'kernel_regularizer_l2':[0.0001],#[0,0.001,0.0001],
    'bias_regularizer':[0.0001],#[0,0.001,0.0001],
    'activity_regularizer':[0.0001],#[0,0.001,0.0001],

#    'batc_normalization':[False,True],
#    'dropout': [0,0.2,0.4],
    'dropout': [0],
    
    'optimizer': ['rmsprop','adam','adadelta','adamax','nadam','adagrad'],
    #'kernel_initializer': ['orthogonal','identity','zeros','ones','uniform'],
    'kernel_initializer': ['orthogonal'],
    #'activation_layer':['sigmoid','tanh','selu','elu','relu'],
    'activation_layer':['relu'],
    #'batc_normalization':[False,True]
    'batc_normalization':[False],
    'alpha':[0.85],
    'gamma':[1.7]
    
    
    
}

In [160]:

def numerai_model(x_train, y_train, x_val, y_val, params):
    
    print(params)

    model = Sequential()
    
    ## initial layer
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation='relu',
               
                    kernel_initializer = params['kernel_initializer'],
                    kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                                l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                   ))
    if params['batc_normalization']==True:
        model.add(BatchNormalization())
        
    if params['dropout']!=0:
        model.add(Dropout(params['dropout']))
    
    ## hidden layers
    for i in range(params['hidden_layers']):
        print (f"adding layer {i+1}")
        model.add(Dense(params['hidden_neuron'], activation='relu',
                    kernel_initializer=params['kernel_initializer'],
                        kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                                    l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                       ))
        if params['batc_normalization']==True:
            model.add(BatchNormalization())
            
        if params['dropout']!=0:
            model.add(Dropout(params['dropout']))
    
    
    ## final layer
    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer'],
                   kernel_regularizer=keras.regularizers.l1_l2(l1=params['kernel_regularizer_l1'],
                                                               l2=params['kernel_regularizer_l2']),
                    bias_regularizer=keras.regularizers.l2(params['bias_regularizer']),
                    activity_regularizer=keras.regularizers.l2(params['activity_regularizer'])
                   ))
    
    model.compile(loss=[binary_focal_loss(alpha=params['alpha'], gamma=params['gamma'])],
                  optimizer=params['optimizer'],#tf.keras.optimizers.Adamax(learning_rate=params['lr']),
                  metrics=[tfa.metrics.FBetaScore(num_classes=1, beta=1.0, threshold=0.5),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall')])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=0,
                        callbacks = [
                                     EarlyStopping(monitor='val_loss',
                                        min_delta=0.01,
                                        patience=50, mode='min',verbose=1,
                                                      restore_best_weights=True)
                                    ] #,ta.live(),
                        )

    return history, model

In [161]:
t = ta.Scan(x=train_df, y=train_label,
            x_val=test_df, y_val=test_label,
            model=numerai_model,
            params=p,  
            experiment_name='Predykcja klasy M - podejscie z Focal-loss (nowe)')



  0%|                                                                                           | 0/72 [00:00<?, ?it/s]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.85, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 50, 'gamma': 1.7, 'hidden_layers': 1, 'hidden_neuron': 50, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'optimizer': 'rmsprop'}
adding layer 1
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the funct

  6%|█████▏                                                                            | 1/16 [01:52<28:09, 112.64s/it]


  1%|█▏                                                                                 | 1/72 [00:05<06:36,  5.59s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.85, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 50, 'gamma': 1.7, 'hidden_layers': 1, 'hidden_neuron': 50, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'optimizer': 'adam'}
adding layer 1
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function



  3%|██▎                                                                                | 2/72 [00:11<06:30,  5.57s/it]

...



 97%|███████████████████████████████████████████████████████████████████████████████▋  | 70/72 [06:56<00:14,  7.25s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.85, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 1.7, 'hidden_layers': 5, 'hidden_neuron': 150, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'optimizer': 'nadam'}
adding layer 1
adding layer 2
adding layer 3
adding layer 4
adding layer 5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribu



 99%|████████████████████████████████████████████████████████████████████████████████▊ | 71/72 [07:04<00:07,  7.43s/it]

{'activation_layer': 'relu', 'activity_regularizer': 0.0001, 'alpha': 0.85, 'batc_normalization': False, 'batch_size': 32, 'bias_regularizer': 0.0001, 'dropout': 0, 'epochs': 100000, 'first_neuron': 110, 'gamma': 1.7, 'hidden_layers': 5, 'hidden_neuron': 150, 'kernel_initializer': 'orthogonal', 'kernel_regularizer_l1': 0.0001, 'kernel_regularizer_l2': 0.0001, 'last_activation': 'sigmoid', 'optimizer': 'adagrad'}
adding layer 1
adding layer 2
adding layer 3
adding layer 4
adding layer 5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attri



100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [07:10<00:00,  5.98s/it]


In [162]:
df = pd.read_csv('D:/STUDIA/ROK_II/Magisterka/Modele/Dane pierwotne/M/Predykcja klasy M - podejscie z Focal-loss (nowe)/050522225017.csv')

In [163]:
df

,round_epochs,loss,fbeta_score,precision,recall,val_loss,val_fbeta_score,val_precision,val_recall,activation_layer,...,epochs,first_neuron,gamma,hidden_layers,hidden_neuron,kernel_initializer,kernel_regularizer_l1,kernel_regularizer_l2,last_activation,optimizer
0,93,0.051822,[0.2993763],0.182278,0.837209,0.064037,[0.28571427],0.169492,0.909091,relu,...,100000,50,1.7,1,50,orthogonal,0.0001,0.0001,sigmoid,rmsprop
1,96,0.051066,[0.30852994],0.182796,0.988372,0.065722,[0.28571427],0.169492,0.909091,relu,...,100000,50,1.7,1,50,orthogonal,0.0001,0.0001,sigmoid,adam
2,51,0.144740,[0.24305557],0.142857,0.813953,0.145879,[0.18348624],0.114943,0.454545,relu,...,100000,50,1.7,1,50,orthogonal,0.0001,0.0001,sigmoid,adadelta
3,158,0.054593,[0.317757],0.189310,0.988372,0.059225,[0.30612242],0.197368,0.681818,relu,...,100000,50,1.7,1,50,orthogonal,0.0001,0.0001,sigmoid,adamax
4,97,0.051204,[0.30711612],0.183036,0.953488,0.059571,[0.37837836],0.466667,0.318182,relu,...,100000,50,1.7,1,50,orthogonal,0.0001,0.0001,sigmoid,nadam
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,70,0.056006,[0.26299694],0.151408,1.000000,0.056523,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adam
68,51,0.935253,[0.],0.000000,0.000000,0.936036,[0.],0.000000,0.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
69,108,0.056058,[0.26299694],0.151408,1.000000,0.056570,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adamax
70,66,0.057568,[0.26299694],0.151408,1.000000,0.058078,[0.26829267],0.154930,1.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,nadam


In [164]:
stare_wart=df['val_fbeta_score']
nowe_wart=[]
for x in stare_wart:
    wart=x.replace('[','')
    wart=wart.replace(']','')
    wart=float(wart)
    nowe_wart.append(wart)

In [165]:
df['val_fbeta_score']=nowe_wart

In [166]:
df=df.sort_values('val_loss',ascending=False)

In [167]:
df

,round_epochs,loss,fbeta_score,precision,recall,val_loss,val_fbeta_score,val_precision,val_recall,activation_layer,...,epochs,first_neuron,gamma,hidden_layers,hidden_neuron,kernel_initializer,kernel_regularizer_l1,kernel_regularizer_l2,last_activation,optimizer
68,51,0.935253,[0.],0.000000,0.000000,0.936036,0.000000,0.000000,0.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
71,51,0.929338,[0.27633852],0.162272,0.930233,0.929752,0.279720,0.165289,0.909091,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
32,51,0.807648,[0.04347826],0.057692,0.034884,0.808121,0.102564,0.117647,0.090909,relu,...,100000,50,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
35,51,0.803775,[0.28832117],0.170996,0.918605,0.804499,0.266667,0.159292,0.818182,relu,...,100000,50,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
59,51,0.607745,[0.28169015],0.165975,0.930233,0.608829,0.281690,0.166667,0.909091,relu,...,100000,110,1.7,3,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,108,0.056058,[0.26299694],0.151408,1.000000,0.056570,0.268293,0.154930,1.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adamax
67,70,0.056006,[0.26299694],0.151408,1.000000,0.056523,0.268293,0.154930,1.000000,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adam
31,62,0.055865,[0.26299694],0.151408,1.000000,0.056383,0.268293,0.154930,1.000000,relu,...,100000,50,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adam
33,98,0.055828,[0.26299694],0.151408,1.000000,0.056350,0.268293,0.154930,1.000000,relu,...,100000,50,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adamax


In [168]:
df=df[df['recall']!=1]

In [169]:
df=df[df['precision']!=1]

In [173]:
df=df[df['val_fbeta_score']!=0]

In [174]:
df

,round_epochs,loss,fbeta_score,precision,recall,val_loss,val_fbeta_score,val_precision,val_recall,activation_layer,...,epochs,first_neuron,gamma,hidden_layers,hidden_neuron,kernel_initializer,kernel_regularizer_l1,kernel_regularizer_l2,last_activation,optimizer
71,51,0.929338,[0.27633852],0.162272,0.930233,0.929752,0.279720,0.165289,0.909091,relu,...,100000,110,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
32,51,0.807648,[0.04347826],0.057692,0.034884,0.808121,0.102564,0.117647,0.090909,relu,...,100000,50,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
35,51,0.803775,[0.28832117],0.170996,0.918605,0.804499,0.266667,0.159292,0.818182,relu,...,100000,50,1.7,5,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
59,51,0.607745,[0.28169015],0.165975,0.930233,0.608829,0.281690,0.166667,0.909091,relu,...,100000,110,1.7,3,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
56,51,0.608901,[0.26966295],0.156425,0.976744,0.608227,0.281879,0.165354,0.954545,relu,...,100000,110,1.7,3,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
20,51,0.482894,[0.262697],0.154639,0.872093,0.483054,0.203390,0.162162,0.272727,relu,...,100000,50,1.7,3,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
23,51,0.480928,[0.28368795],0.167364,0.930233,0.481366,0.275362,0.163793,0.863636,relu,...,100000,50,1.7,3,150,orthogonal,0.0001,0.0001,sigmoid,adagrad
65,51,0.346391,[0.23433243],0.153025,0.500000,0.347060,0.179775,0.119403,0.363636,relu,...,100000,110,1.7,5,50,orthogonal,0.0001,0.0001,sigmoid,adagrad
44,51,0.288262,[0.08633094],0.113208,0.069767,0.289086,0.057143,0.076923,0.045455,relu,...,100000,110,1.7,1,150,orthogonal,0.0001,0.0001,sigmoid,adadelta
50,51,0.287249,[0.26153845],0.150709,0.988372,0.287565,0.268293,0.154930,1.000000,relu,...,100000,110,1.7,3,50,orthogonal,0.0001,0.0001,sigmoid,adadelta
